# How Does Pharmaceutical Price in Denmark Changed From 2015

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which will be used multiple times in this notebook.

Imports and set magics:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib_venn import venn2 # install with pip install matplotlib-venn

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

# Read and clean data

## Pharmaceutical Price Data

**Read the employment data** in ``project1.xlsx`` and **clean it** removing and renaming columns:

In [2]:
# a. load
phar = pd.read_excel('project1.xlsx')
phar.head(5)

,ATC,Medicines,Number,Packing,Strength,Form,Firm,Indicators,20150202,20150216,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
0,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AIP,407.36,407.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP,570.25,570.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01AA01,Bifluorid,43158,10 g,NaN,dentalsuspension,Voco,AIP,602.07,602.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# b. drop columns
drop_these = ['Packing', 'Strength','Form','Firm']
phar.drop(drop_these, axis=1, inplace=True)

The dataset now looks like this:

In [4]:
phar.head(5)

,ATC,Medicines,Number,Indicators,20150202,20150216,20150302,20150316,20150330,20150413,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
0,A01AA01,Bifluorid,42846,AIP,407.36,407.36,407.36,407.36,407.36,407.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01AA01,Bifluorid,42846,AUP,570.25,570.25,570.25,570.25,570.25,570.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01AA01,Bifluorid,42846,DDD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01AA01,Bifluorid,42846,AUP_pr_DDD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01AA01,Bifluorid,43158,AIP,602.07,602.07,602.07,602.07,602.07,602.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Remove all rows which do not contain full time series data**:

In [5]:
phar.dropna(inplace = True)
phar.head(10)

,ATC,Medicines,Number,Indicators,20150202,20150216,20150302,20150316,20150330,20150413,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
12,A01AA01,Duraphat,66411,AIP,40.9900,40.9900,40.9900,40.9900,40.9900,40.9900,...,46.7100,45.2300,45.2300,44.1200,44.120,44.120,42.950,41.840,41.840,40.170
13,A01AA01,Duraphat,66411,AUP,69.7000,69.7000,69.7000,69.7000,69.7000,69.7000,...,70.0000,68.0000,68.0000,66.5000,66.350,66.350,64.750,63.250,63.250,61.000
16,A01AA01,Duraphat,71589,AIP,193.7000,193.7000,193.7000,193.7000,193.7000,193.7000,...,193.7000,193.7000,193.7000,193.7000,193.700,195.130,195.130,195.130,195.130,195.130
17,A01AA01,Duraphat,71589,AUP,278.3500,278.3500,278.3500,278.3500,278.3500,278.3500,...,268.8000,268.8000,268.8000,268.8000,268.100,270.000,270.000,270.000,270.000,270.000
20,A01AA01,Duraphat,416859,AIP,193.7000,193.7000,193.7000,193.7000,193.7000,193.7000,...,193.1100,192.0000,192.0000,190.8900,190.890,195.130,194.010,190.680,190.680,189.600
21,A01AA01,Duraphat,416859,AUP,278.3500,278.3500,278.3500,278.3500,278.3500,278.3500,...,268.0000,266.5000,266.5000,265.0000,264.300,270.000,268.500,264.000,264.000,262.550
36,A01AB09,Brentan,50963,AIP,40.5300,40.5300,40.5300,40.5300,40.5300,40.5300,...,30.8500,30.8500,30.8500,30.8500,35.480,35.480,35.480,35.480,35.480,35.480
37,A01AB09,Brentan,50963,AUP,69.0500,69.0500,69.0500,69.0500,69.0500,69.0500,...,48.5500,48.5500,48.5500,48.5500,54.700,54.700,54.700,54.700,54.700,54.700
38,A01AB09,Brentan,50963,DDD,4.0000,4.0000,4.0000,4.0000,4.0000,4.0000,...,4.0000,4.0000,4.0000,4.0000,4.000,4.000,4.000,4.000,4.000,4.000
39,A01AB09,Brentan,50963,AUP_pr_DDD,17.2625,17.2625,17.2625,17.2625,17.2625,17.2625,...,12.1375,12.1375,12.1375,12.1375,13.675,13.675,13.675,13.675,13.675,13.675


**Keep observations with indicators = AIP, AUP_pr_DDD, DDD**:

In [6]:
I = phar.Indicators.str.contains('AIP')
phar = phar.loc[I == True]
phar.head(10)

,ATC,Medicines,Number,Indicators,20150202,20150216,20150302,20150316,20150330,20150413,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
12,A01AA01,Duraphat,66411,AIP,40.99,40.99,40.99,40.99,40.99,40.99,...,46.71,45.23,45.23,44.12,44.12,44.12,42.95,41.84,41.84,40.17
16,A01AA01,Duraphat,71589,AIP,193.70,193.70,193.70,193.70,193.70,193.70,...,193.70,193.70,193.70,193.70,193.70,195.13,195.13,195.13,195.13,195.13
20,A01AA01,Duraphat,416859,AIP,193.70,193.70,193.70,193.70,193.70,193.70,...,193.11,192.00,192.00,190.89,190.89,195.13,194.01,190.68,190.68,189.60
36,A01AB09,Brentan,50963,AIP,40.53,40.53,40.53,40.53,40.53,40.53,...,30.85,30.85,30.85,30.85,35.48,35.48,35.48,35.48,35.48,35.48
48,A01AD02,Andolex,484618,AIP,73.00,73.00,73.00,73.00,73.00,73.00,...,62.50,61.05,59.92,55.20,55.20,52.00,46.55,46.55,46.55,46.55
60,A01AD11,Dolodent,56036,AIP,77.62,77.62,77.62,77.62,77.62,77.62,...,83.83,83.83,83.83,83.83,83.83,83.83,83.83,83.83,83.83,83.83
68,A02BB01,Cytotec,3455,AIP,141.67,141.67,141.67,141.67,141.67,141.67,...,178.50,178.50,178.50,178.50,178.50,178.50,178.50,178.50,178.50,178.50
72,A02BC01,Losec,488627,AIP,662.41,662.41,662.41,662.41,662.41,662.41,...,662.41,662.41,662.41,662.41,662.41,662.41,662.41,662.41,662.41,662.41
164,A02BC01,"Omeprazol ""Medical Valley""",555860,AIP,50.10,50.10,50.10,50.10,50.10,50.10,...,150.00,147.65,151.50,150.50,151.95,151.95,150.36,144.80,136.05,121.63
168,A02BC01,"Omeprazol ""Medical Valley""",569422,AIP,102.00,102.00,102.00,102.00,102.00,102.00,...,305.00,305.00,305.00,305.00,305.00,305.00,305.00,298.64,290.04,265.54


In [9]:
phar.describe()

,Number,20150202,20150216,20150302,20150316,20150330,20150413,20150427,20150511,20150525,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
count,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,...,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000,3966.000000
mean,222258.166919,2090.119997,2089.856846,2087.317237,2085.033164,2084.152446,2082.571967,2081.074932,2082.562907,2080.755754,...,2001.584806,1986.751152,1983.190827,1983.867242,1983.974796,1980.755348,1980.348628,1979.806394,1979.204433,1978.859168
std,202988.474132,8789.763462,8790.768544,8790.543519,8790.779761,8790.715410,8790.755172,8790.483129,8790.200421,8790.216111,...,8398.711390,8305.555741,8294.718313,8294.973587,8294.487168,8290.958610,8291.204575,8291.271722,8291.318998,8291.215241
min,5.000000,2.800000,2.800000,2.800000,2.800000,2.650000,2.650000,2.650000,3.090000,3.090000,...,3.340000,3.340000,3.500000,3.500000,4.750000,3.750000,3.750000,3.750000,3.750000,3.750000
25%,50906.750000,88.225000,86.470000,86.025000,85.000000,84.937500,85.187500,87.585000,88.000000,86.782500,...,94.677500,95.000000,95.425000,96.287500,98.000000,97.677500,97.917500,96.400000,95.050000,95.200000
50%,129976.000000,275.590000,274.955000,274.630000,273.290000,273.920000,269.550000,268.260000,270.285000,271.330000,...,271.745000,272.745000,273.430000,274.955000,275.000000,274.955000,272.450000,270.365000,271.735000,271.330000
75%,435661.500000,992.085000,995.520000,991.045000,987.975000,985.175000,978.190000,977.925000,985.332500,977.097500,...,931.440000,936.517500,937.370000,937.580000,938.145000,937.370000,937.580000,936.277500,934.552500,937.370000
max,599882.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,...,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000,186800.000000


**Convert the dataset to long format**:

In [8]:
# a. rename year columns
dict0 = {str(i):f'price{i}' for i in range(20150202,20200323)}
phar.rename(columns = dict0, inplace=True)

# b. convert to long
phar_long = pd.wide_to_long(phar, stubnames='price', i='Number').reset_index()

# c. show
phar_long.head(10)

TypeError: wide_to_long() missing 1 required positional argument: 'j'

## Income data

**Read the income data** in ``INDKP101.xlsx`` and **clean it**:

In [ ]:
# a. load
inc = pd.read_excel('INDKP101.xlsx', skiprows=2)

# b. drop and rename columns
inc.drop([f'Unnamed: {i}' for i in range(3)], axis=1, inplace=True)
inc.rename(columns = {'Unnamed: 3':'municipality'}, inplace=True)

# c. drop rows with missing
inc.dropna(inplace=True)

# d. remove non-municipalities
inc = dataproject.only_keep_municipalities(inc)

# e. convert to long
inc.rename(columns = {str(i):f'income{i}' for i in range(1986,2018)}, inplace=True)
inc_long = pd.wide_to_long(inc, stubnames='income', i='municipality', j='year').reset_index()

# f. show
inc_long.head(5)

> **Note:** The function ``dataproject.only_keep_municipalities()`` is used on both the employment and the income datasets.

## Explore data set

In order to be able to **explore the raw data**, we here provide an **interactive plot** to show, respectively, the employment and income level in each municipality

The **static plot** is:

In [ ]:
def plot_empl_inc(empl,inc,dataset,municipality): 
    
    if dataset == 'Employment':
        df = empl
        y = 'employment'
    else:
        df = inc
        y = 'income'
    
    I = df['municipality'] == municipality
    ax = df.loc[I,:].plot(x='year', y=y, style='-o')

The **interactive plot** is:

In [ ]:
widgets.interact(plot_empl_inc, 
    
    empl = widgets.fixed(empl_long),
    inc = widgets.fixed(inc_long),
    dataset = widgets.Dropdown(description='Dataset', 
                               options=['Employment','Income']),
    municipality = widgets.Dropdown(description='Municipality', 
                                    options=empl_long.municipality.unique())
                 
); 

ADD SOMETHING HERE IF THE READER SHOULD KNOW THAT E.G. SOME MUNICIPALITY IS SPECIAL.

# Merge data sets

We now create a data set with **municpalities which are in both of our data sets**. We can illustrate this **merge** as:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('inc', 'empl'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

In [ ]:
merged = pd.merge(empl_long, inc_long, how='inner',on=['municipality','year'])

print(f'Number of municipalities = {len(merged.municipality.unique())}')
print(f'Number of years          = {len(merged.year.unique())}')

# Analysis

To get a quick overview of the data, we show some **summary statistics by year**:

In [ ]:
merged.groupby('year').agg(['mean','std']).round(2)

ADD FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.